In [ ]:
# using Pkg
# Pkg.activate(".")
# Pkg.add("WordTokenizers")
# Pkg.add("Languages")
# Pkg.instantiate()
# Pkg.add("SQLite")
# Pkg.add("DBInterface")
# Pkg.add("JSON3")
# Pkg.add("EasyConfig")
# Pkg.add("MurmurHash3")
# Pkg.add("TextAnalysis")
# Pkg.add("UUIDs")
# Pkg.add("PooledArrays")
include("src/lisa_store.jl")

using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using EasyConfig
using HDF5


# Domains and co-domains

Here we'll try to reduce ambiguity of tracing tokens from the HllSet presentation.  

The consistency of conversion of datasets to HllSets depends on two factors:

 1. The $p$ parameter that defines the precision of the conversion through the number of bins;
 2. The type of a hash function that we are using.

Specifically, the output from hash function depends on the seed values used in initiating of the hash function. Applying different seed values we can control the generated hashes.

Here is an idea of the algorithm:

 1. We are performing the dataset processing as usual, utilizing standard hash function:

$$F_{(std)}: X_{(std)} \to Y_{(std)}$$

 2. Then we are tracing original tokens by applying back processing:

$$G_{(std)}: Y_{(std)} \to X_{(std)}$$

 3. Now we can perform the same dataset processing using the same hash function but with different seed values:

$$F_{(seed)}: X_{(seed)} \to Y_{(seed)}$$

 4. And we will trace back the results from modified hash function:

$$G_{(seed)}: Y_{(seed)} \to X_{(seed)}$$

It is possible, that

 $$X_{(seed)} \not= X_{(std)}$$

But it is also obvious, that tokens from the original dataset should be in both results.

## Standard processing


In [ ]:
db = Graph.DB("lisa_analytics.db")
db.sqlitedb

In [11]:
Store.book_file(db, "/home/alexmy/JULIA/DEMO/sample/")

sha1: 0b90b1fee69c77ffa3efe57db7788112ef96dba6
sha1: 

6be12bee4edf7c96016907e44bb520be80dc9232


In [12]:
uuid = string(uuid4())
df = Graph.set_lock!(db, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", 
    uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row) 
    col_uuid = string(uuid4())
    Store.ingest_csv_by_column(db, assign, col_uuid; limit=10000, offset=10)
end

Processed column: 8


Processed column: 15


In [13]:
ds_id = "3f9526f8d331b9519b8632a11b2d344ab7c647b6"
node = Graph.getnode(db, ds_id, :; table_name="t_nodes")

Node(3f9526f8d331b9519b8632a11b2d344ab7c647b6; ["csv_column"]; props: column_name="Vehicle type", file_sha1="0b90b1fee69c77ffa3efe57db7788112ef96dba6", column_type="String")

In [14]:
tokens = Store.collect_tokens(db, "3f9526f8d331b9519b8632a11b2d344ab7c647b6")
println(tokens)

Set(["Taxi", "mgw", "Taxi/Private", "Moped", "String", "M/cycle", "vehicle", "and", "seats", "over", "coach", "more", "Motor", "pass", "Minibus", "horse", "motor", "under", "hire", "cycle", "Motorcycle", "Private", "Vehicle", "Other", "Goods", "tonnes", "Bus", "gross", "Van", "Ridden", "Car", "weight", "car", "passenger", "from", "type", "maximum", "Pedal"])


## Processing with seeded hash

We'll go through the same steps with the same params except the database.
 - the db name would be "db_seed.db"
  

In [15]:
db_seed = Graph.DB("db_seed.db")

Graph.DB("db_seed.db") (34 assignments, 0 commits, 166 tokens, 0 nodes, 0 edges25 t_nodes, 23 t_edges)

In [16]:
Store.book_file(db_seed, "/home/alexmy/JULIA/DEMO/sample/"; seed=42, P=10)

sha1: 0b90b1fee69c77ffa3efe57db7788112ef96dba6
sha1: 

6be12bee4edf7c96016907e44bb520be80dc9232


In [18]:
uuid = string(uuid4())
df = Graph.set_lock!(db_seed, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", 
    uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row) 
    col_uuid = string(uuid4())
    # Important, do not forget to set HllSet precission parameter p to 8
    Store.ingest_csv_by_column(db_seed, assign, col_uuid; limit=10000, offset=10, p=10, seed=42)
end

### We are getting the dataset directly from **nodes** table of the "db_seed.db" database.

We are utilizing the fact that SHA1 node ID is not affected by changing the hash function for the tokens encoding.

In [19]:
ds_id = "3f9526f8d331b9519b8632a11b2d344ab7c647b6"
node_seed = Graph.getnode(db_seed, ds_id, :; table_name="t_nodes")

Node(3f9526f8d331b9519b8632a11b2d344ab7c647b6; ["csv_column"]; props: column_name="Vehicle type", file_sha1="0b90b1fee69c77ffa3efe57db7788112ef96dba6", column_type="String")

In [20]:
tokens_seed = Store.collect_tokens(db_seed, "3f9526f8d331b9519b8632a11b2d344ab7c647b6")
println(tokens_seed)

Set(["maximum", "Pedal", "Taxi/Private", "Moped", "M/cycle", "String", "over", "and", "seats", "vehicle", "coach", "more", "Minibus", "pass", "Motor", "horse", "hire", "motor", "Vehicle", "cycle", "Goods", "Motorcycle", "Private", "Other", "under", "tonnes", "Bus", "gross", "Van", "Ridden", "Car", "weight", "car", "passenger", "from", "type", "mgw", "Taxi"])


In [21]:
intersection = intersect(tokens, tokens_seed)

Set{String} with 38 elements:
  "mgw"
  "Taxi"
  "Taxi/Private"
  "Moped"
  "M/cycle"
  "String"
  "vehicle"
  "and"
  "seats"
  "over"
  "coach"
  "more"
  "Minibus"
  "pass"
  "Motor"
  "horse"
  "hire"
  "motor"
  "Vehicle"
  ⋮ 

In [22]:
println("tokens size: ", length(tokens), 
    ";\ntokens_seed size: ", length(tokens_seed), 
    ";\nintersection size: ", length(intersection))

tokens size: 38;
tokens_seed size: 38;
intersection size: 38


### Lets check how use of a seeded hash affected HllSets

In [23]:
hll_std = SetCore.HllSet{10}()
hll_seed = SetCore.HllSet{10}()

dataset_std = node.dataset
dataset_seed = node_seed.dataset

println("dataset_std size: ", length(dataset_std), 
    ";\ndataset_seed size: ", length(dataset_seed))

# Restore collect_hll_sets
hll_std = SetCore.restore(hll_std, Vector{UInt64}(dataset_std))
hll_seed = SetCore.restore(hll_seed, Vector{UInt64}(dataset_seed))

println("hll_std size: ", SetCore.count(hll_std), 
    ";\nhll_seed size: ", SetCore.count(hll_seed))

hll_intersection = intersect(hll_std, hll_seed)
# SetCore.count(hll_intersection)

println("hll_intersection size: ", SetCore.count(hll_intersection))

dataset_std size: 1024;
dataset_seed size: 1024
hll_std size: 36;
hll_seed size: 36
hll_intersection size: 1


### So, we are lucky , we got not empty intersection from two HllSets built using different hash functions. (Or may be not, because $1$ is small and could be within the range of an estimation error)

We also can see that the cardinality estimations in our case are not bad. The difference in both case is equal $2$, or about $2.63$%.

# Applying HllSets for Tabular data structures

In [1]:
include("src/lisa_store.jl")

using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using EasyConfig
using SparseArrays

In [2]:
db = Graph.DB("lisa_analytics.db")

Graph.DB("lisa_analytics.db") (34 assignments, 0 commits, 174 tokens, 0 nodes, 0 edges125 t_nodes, 123 t_edges)

In [ ]:
Store.book_file(db, "/home/alexmy/JULIA/DEMO/sample/"; column=false)

In [ ]:
uuid = string(uuid4())
df = Graph.set_lock!(db, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", 
    uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row) 
    col_uuid = string(uuid4())    
    Store.ingest_csv_by_row(db, assign; limit=50, offset=10)
end

In [9]:
function get_joined_nodes(db::Graph.DB, source_id::String, r_type::String)
    col_query = """
    SELECT *
    FROM t_nodes
    INNER JOIN t_edges ON t_edges.target = t_nodes.sha1
    WHERE t_edges.source LIKE '%$source_id%' AND t_edges.r_type LIKE '%$r_type%';
    """
    return DBInterface.execute(db.sqlitedb, col_query) |> DataFrame
end

get_joined_nodes (generic function with 1 method)

In [10]:
function get_card_matrix(db::Graph.DB, source_id::String)
    
    row_nodes = get_joined_nodes(db, source_id, "has_row")
    row_sets = [SetCore.restore(SetCore.HllSet{10}(), JSON3.read(row.dataset, Vector{UInt64})) for row in eachrow(row_nodes)]

    # Select column nodes
    column_nodes = get_joined_nodes(db, source_id, "has_column")
    column_sets = [SetCore.restore(SetCore.HllSet{10}(), JSON3.read(row.dataset, Vector{UInt64})) for row in eachrow(column_nodes)]

    # Create a matrix where each cell is the cardinality of the intersection of the corresponding row and column HllSet
    matrix = spzeros(length(row_sets), length(column_sets))
    for (i, row_set) in enumerate(row_sets)
        for (j, column_set) in enumerate(column_sets)
            matrix[i, j] = SetCore.count(SetCore.intersect(row_set, column_set))
        end
    end

    return matrix
end

source_id = "0b90b1fee69c77ffa3efe57db7788112ef96dba6"
matrix = get_card_matrix(db, source_id)

for row in eachrow(matrix)
    println(row)
end

[2.0, 2.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 2.0, 1.0, 1.0, 2.0, 4.0, 5.0, 4.0]
[2.0, 3.0, 2.0, 1.0, 1.0, 4.0, 4.0, 4.0]
[2.0, 2.0, 2.0, 1.0, 1.0, 4.0, 4.0, 3.0]
[2.0, 2.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 1.0, 4.0, 3.0, 3.0]
[2.0, 2.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 2.0, 1.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 2.0, 4.0, 4.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 2.0, 4.0, 3.0, 4.0]
[2.0, 3.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 2.0, 4.0, 3.0, 4.0]
[2.0, 3.0, 1.0, 1.0, 2.0, 4.0, 4.0, 3.0]
[2.0, 3.0, 1.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 1.0, 4.0, 4.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 2.0, 4.0, 4.0, 3.0]
[2.0, 2.0, 1.0, 1.0, 1.0, 4.0, 8.0, 3.0]
[2.0, 2.0, 1.0, 1.0, 1.0, 4.0, 3.0, 3.0]
[2.0, 2.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 2.0, 2.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 1.0, 1.0, 2.0, 4.0, 3.0, 3.0]
[2.0, 3.0, 1.0, 